In [2]:

from flax import linen as nn
import numpy as np
from typing import Any, Callable, Optional, Tuple, Type, List
from jax import lax, random, numpy as jnp
import einops
import torchio
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import jax
import tensorflow as tf
import torch 
import einops
import torchio as tio
import optax
from flax.training import train_state  
from torch.utils.data import DataLoader
import jax.profiler
import ml_collections
from ml_collections import config_dict
from functools import partial
import toolz
import chex    


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:



def filter_mask_of_intrest(masks,shift_x,shift_y):
  masks= jnp.round(masks)
  masks_a=(masks[:,:,0])==shift_x
  masks_b=(masks[:,:,1])==shift_y
  mask_0= jnp.logical_and(masks_a,masks_b).astype(int)    
  return mask_0




def get_new_mask_from_probs(mask_old,bi_chan_probs,dim_stride,mask_shape,mask_shape_end,rearrange_to_intertwine_einops ):
    """ 
    given bi channel probs where first channel will be interpreted as probability of being the same id (1 or 0)
    as the sv backward the axis; and second channel probability of being the same supervoxel forward the axis 
    """
    #we add 1 becouse of batch
    dim_stride_curr=dim_stride+1
    #we get propositions forward and backward od old mask
    old_forward=jnp.take(mask_old, indices=jnp.arange(1,mask_shape[dim_stride_curr]),axis=dim_stride_curr )
    old_back =jnp.take(mask_old, indices=jnp.arange(0,mask_shape[dim_stride_curr]),axis=dim_stride_curr )
    #in order to get correct shape we need to add zeros to forward
    to_end_grid=jnp.ones(mask_shape_end)
    old_forward= jnp.concatenate((old_forward,to_end_grid) ,axis= dim_stride_curr)

    
    old_propositions=jnp.stack([old_forward,old_back],axis=-1)# w h n_dim 2
    #chosen values and its alternative
    bi_chan_probs=jnp.round(bi_chan_probs)# TODO change into diff round - this is non differentiable !!   

    bi_chan_probs=einops.repeat(bi_chan_probs,'w h pr->w h d pr',d=2)

    chosen_values=jnp.multiply(old_propositions,bi_chan_probs)
    chosen_values= jnp.sum(chosen_values,axis=-1)
    chosen_values_alt=jnp.multiply(old_propositions,(jnp.flip(bi_chan_probs,axis=-1)))
    chosen_values_alt= jnp.sum(chosen_values_alt,axis=-1)

    
    mask_combined=einops.rearrange([chosen_values,mask_old],rearrange_to_intertwine_einops) 
    mask_combined_alt=einops.rearrange([chosen_values_alt,mask_old],rearrange_to_intertwine_einops)

    shift_x=0
    shift_y=0
    mask_combined_print=filter_mask_of_intrest(jnp.round(mask_combined[0,:,:,:]),shift_x,shift_y)
    mask_old_print=filter_mask_of_intrest(jnp.round(mask_old[0,:,:,:]),shift_x,shift_y)
    chosen_values_print=filter_mask_of_intrest(jnp.round(chosen_values[0,:,:,:]),shift_x,shift_y)
    # jax.debug.print("{y} {z} \n mask_combined_print \n {a} \n mask_old_print \n{b} \n chosen_values_print \n {c} \n *****************", a=mask_combined_print ,b=mask_old_print, c=chosen_values_print ,y=shift_x, z= shift_y)

    return mask_combined,mask_combined_alt
dim_stride=0
rearrange_to_intertwine_einops='f h w cc-> (h f) w cc'

get_new_mask_from_probs(mask_old,bi_chan_probs,dim_stride,mask_shape,mask_shape_end,rearrange_to_intertwine_einops )

NameError: name 'mask_old' is not defined

In [ ]:
def harder_diff_round(a):
    return jnp.round(a)

def differentiable_eq(a:float,b:float):
    """
    will give big value if a anb b are similar and small otherwise
    bot a and b are assumed to be between 0 and 1
    """
    a= harder_diff_round(a)
    b= harder_diff_round(b)
    res=a*b+(1-a)*(1-b)
    return harder_diff_round(res)

def differentiable_and(a:float,b:float):
    """
    will give big value if a anb b are similar and small otherwise
    bot a and b are assumed to be between 0 and 1
    """
    a= harder_diff_round(a)
    b= harder_diff_round(b)
    res=a*b
    return harder_diff_round(res)



#versions with second entry keeping as int
v_differentiable_eq=jax.vmap(differentiable_eq,in_axes=(0,None))
v_v_differentiable_eq=jax.vmap(v_differentiable_eq,in_axes=(0,None))
v_v_v_differentiable_eq=jax.vmap(v_v_differentiable_eq,in_axes=(0,None))

#version where both entries are 3 dimensional
v_differentiable_eq_bi=jax.vmap(differentiable_eq,in_axes=(0,0))
v_v_differentiable_eq_bi=jax.vmap(v_differentiable_eq_bi,in_axes=(0,0))
v_v_v_differentiable_eq_bi=jax.vmap(v_v_differentiable_eq_bi,in_axes=(0,0))

def filter_mask_of_intrest(mask,shift_x,shift_y):
    """
    filters the mask to set to 1 only if it is this that we are currently intressted in 

    """
    coor_0_agree=v_v_differentiable_eq(mask[:,:,0],shift_x)
    coor_1_agree=v_v_differentiable_eq(mask[:,:,1],shift_y)
    print(f"coor_0_agree {coor_0_agree} coor_1_agree {coor_1_agree} ")
    return differentiable_and(coor_0_agree,coor_1_agree)  
  
dummy_mask=jnp.array([[[0.0,0.0],[0.0,1.0],[1.0,0.0],[1.0,1.0]]])

filter_mask_of_intrest(dummy_mask, 0,1 )

coor_0_agree [[1. 1. 0. 0.]] coor_1_agree [[0. 1. 0. 1.]] 


Array([[0., 1., 0., 0.]], dtype=float32)

In [ ]:
v_v_differentiable_eq_bi(jnp.array([[0.,0.,1.,1.]]),jnp.array([[1.,0.,1.,0.]]))

Array([[0., 1., 1., 0.]], dtype=float32)

In [ ]:
def get_diameter_no_pad(r):
    """
    so every time we have n elements we can get n+ more elements
    so analyzing on single axis
    start from 1 ->1+1+1 =3 good
    start from 3 ->3+3+1=7 good 
    start from 7 ->7+7+1=15 good 
    """
    curr = 1
    for i in range(0,r):
        curr=curr*2+1
    return curr

def get_diameter(r):
    return get_diameter_no_pad(r)+1

get_diameter(3)

16

In [ ]:
import pandas as pd
def get_initial_supervoxel_masks(orig_grid_shape,shift_x,shift_y):
    """
    on the basis of the present shifts we will initialize the masks
    ids of the supervoxels here are implicit based on which mask and what location we are talking about
    """
    initt=np.zeros(orig_grid_shape)
    initt[shift_x::2,shift_y::2,0]=shift_x
    initt[shift_x::2,shift_y::2,1]=shift_y

    initt[shift_x::2,shift_y::4,2]=1
    initt[shift_x::4,shift_y::2,3]=1

    return initt

def disp_to_pandas(probs,shappe ):
    probs_to_disp= einops.rearrange(probs,'w h c-> (w h) c')
    probs_to_disp=np.round(probs_to_disp,1)
    # probs_to_disp=list(map(lambda twoo: f"{twoo[0]} {twoo[1]}",list(probs_to_disp)))
    probs_to_disp=np.array(probs_to_disp)
    probs_to_disp=list(map(lambda twoo: " ".join(list(map(str,twoo))),list(probs_to_disp)))
    probs_to_disp=np.array(probs_to_disp).reshape(shappe)
    return pd.DataFrame(probs_to_disp)

def disp_to_pandas_curr_shape(probs ):
    return disp_to_pandas(probs,(probs.shape[0],probs.shape[1]) )


cfg = config_dict.ConfigDict()
cfg.total_steps=3
cfg.batch_size=1
# cfg.learning_rate=0.00002 #used for warmup with average coverage loss
cfg.learning_rate=0.0001
cfg.num_dim=4
cfg.batch_size_pmapped=np.max([cfg.batch_size//jax.local_device_count(),1])
cfg.img_size = (cfg.batch_size,1,64,64)
# cfg.label_size = (cfg.batch_size,32,32)
cfg.r_x_total= 3
cfg.r_y_total= 3
cfg.orig_grid_shape= (cfg.img_size[2]//2**cfg.r_x_total,cfg.img_size[3]//2**cfg.r_y_total,cfg.num_dim)
cfg.epsilon=0.0000000001 
cfg = ml_collections.FrozenConfigDict(cfg)

initial_masks= jnp.stack([
            get_initial_supervoxel_masks(cfg.orig_grid_shape,0,0),
            get_initial_supervoxel_masks(cfg.orig_grid_shape,1,0),
            get_initial_supervoxel_masks(cfg.orig_grid_shape,0,1),
            get_initial_supervoxel_masks(cfg.orig_grid_shape,1,1)
                ],axis=0)
initial_masks=jnp.sum(initial_masks,axis=0).astype(int) 
disp_to_pandas_curr_shape(initial_masks)


,0,1,2,3,4,5,6,7
0,0 0 1 1,0 1 1 1,0 0 0 1,0 1 0 1,0 0 1 1,0 1 1 1,0 0 0 1,0 1 0 1
1,1 0 1 1,1 1 1 1,1 0 0 1,1 1 0 1,1 0 1 1,1 1 1 1,1 0 0 1,1 1 0 1
2,0 0 1 0,0 1 1 0,0 0 0 0,0 1 0 0,0 0 1 0,0 1 1 0,0 0 0 0,0 1 0 0
3,1 0 1 0,1 1 1 0,1 0 0 0,1 1 0 0,1 0 1 0,1 1 1 0,1 0 0 0,1 1 0 0
4,0 0 1 1,0 1 1 1,0 0 0 1,0 1 0 1,0 0 1 1,0 1 1 1,0 0 0 1,0 1 0 1
5,1 0 1 1,1 1 1 1,1 0 0 1,1 1 0 1,1 0 1 1,1 1 1 1,1 0 0 1,1 1 0 1
6,0 0 1 0,0 1 1 0,0 0 0 0,0 1 0 0,0 0 1 0,0 1 1 0,0 0 0 0,0 1 0 0
7,1 0 1 0,1 1 1 0,1 0 0 0,1 1 0 0,1 0 1 0,1 1 1 0,1 0 0 0,1 1 0 0


In [ ]:
def masks_binary(masks,shift_x,shift_y,n_2,n_3):
    masks_a=(masks[0,:,:,0])==shift_x
    masks_b=(masks[0,:,:,1])==shift_y
    masks_c=(masks[0,:,:,2])==n_2
    masks_d=(masks[0,:,:,3])==n_3
    mask_0_a= jnp.logical_and(masks_a,masks_b)   
    mask_0_b= jnp.logical_and(masks_c,masks_d)
    mask_0= jnp.logical_and(mask_0_a,mask_0_b)
    return mask_0.astype(int) 


def get_summed_num_mask(masks_bin):
    masks_bin=jnp.round(masks_bin)
    res=(
                 masks_binary(masks_bin,0,0,0,0)
                +masks_binary(masks_bin,1,0,0,0)*2 
                +masks_binary(masks_bin,0,1,0,0)*3 
                +masks_binary(masks_bin,1,1,0,0)*4 
                
                +masks_binary(masks_bin,0,0,1,0)*5
                +masks_binary(masks_bin,0,1,1,0)*6
                +masks_binary(masks_bin,1,0,1,0)*7
                +masks_binary(masks_bin,1,1,1,0)*8  
                
                +masks_binary(masks_bin,0,0,0,1)*9
                +masks_binary(masks_bin,0,1,0,1)*10
                +masks_binary(masks_bin,1,0,0,1)*11
                +masks_binary(masks_bin,1,1,0,1)*12
                
                +masks_binary(masks_bin,0,0,1,1)*13
                +masks_binary(masks_bin,0,1,1,1)*14
                +masks_binary(masks_bin,1,0,1,1)*15
                +masks_binary(masks_bin,1,1,1,1)*16)
    return res
    
print(disp_to_pandas_curr_shape(initial_masks))
print(get_summed_num_mask(einops.rearrange(initial_masks,'w h c-> 1 w h c')))


         0        1        2        3        4        5        6        7
0  0 0 1 1  0 1 1 1  0 0 0 1  0 1 0 1  0 0 1 1  0 1 1 1  0 0 0 1  0 1 0 1
1  1 0 1 1  1 1 1 1  1 0 0 1  1 1 0 1  1 0 1 1  1 1 1 1  1 0 0 1  1 1 0 1
2  0 0 1 0  0 1 1 0  0 0 0 0  0 1 0 0  0 0 1 0  0 1 1 0  0 0 0 0  0 1 0 0
3  1 0 1 0  1 1 1 0  1 0 0 0  1 1 0 0  1 0 1 0  1 1 1 0  1 0 0 0  1 1 0 0
4  0 0 1 1  0 1 1 1  0 0 0 1  0 1 0 1  0 0 1 1  0 1 1 1  0 0 0 1  0 1 0 1
5  1 0 1 1  1 1 1 1  1 0 0 1  1 1 0 1  1 0 1 1  1 1 1 1  1 0 0 1  1 1 0 1
6  0 0 1 0  0 1 1 0  0 0 0 0  0 1 0 0  0 0 1 0  0 1 1 0  0 0 0 0  0 1 0 0
7  1 0 1 0  1 1 1 0  1 0 0 0  1 1 0 0  1 0 1 0  1 1 1 0  1 0 0 0  1 1 0 0
[[13 14  9 10 13 14  9 10]
 [15 16 11 12 15 16 11 12]
 [ 5  6  1  3  5  6  1  3]
 [ 7  8  2  4  7  8  2  4]
 [13 14  9 10 13 14  9 10]
 [15 16 11 12 15 16 11 12]
 [ 5  6  1  3  5  6  1  3]
 [ 7  8  2  4  7  8  2  4]]


In [ ]:
def harder_diff_round(a):
    return jnp.round(a)

def differentiable_eq(a:float,b:float):
    """
    will give big value if a anb b are similar and small otherwise
    bot a and b are assumed to be between 0 and 1
    """
    a= harder_diff_round(a)
    b= harder_diff_round(b)
    res=a*b+(1-a)*(1-b)
    return harder_diff_round(res)

def differentiable_and(a:float,b:float):
    a= harder_diff_round(a)
    b= harder_diff_round(b)
    res=a*b
    return harder_diff_round(res)

def filter_mask_of_intrest(mask,shift_x,shift_y):
    """
    filters the mask to set to 1 only if it is this that we are currently intressted in 
    """
    coor_0_agree=v_v_differentiable_eq(mask[:,:,0],shift_x)
    coor_1_agree=v_v_differentiable_eq(mask[:,:,1],shift_y)

    # coor_0_agree=v_v_differentiable_eq(mask[:,:,0],shift_x)
    # coor_1_agree=v_v_differentiable_eq(mask[:,:,1],shift_y)

    return differentiable_and(coor_0_agree,coor_1_agree)    

In [ ]:
import jax
import jax.numpy as jnp

def a(x):
    return jnp.sin(x*(jnp.pi/2))*jnp.sin(x*(jnp.pi/2))

x = jnp.float32(1.0)

print(jax.make_jaxpr(a)(x))
# { lambda ; a:f32[]. let b:f32[] = sin a in (b,) }

print(jax.make_jaxpr(jax.grad(a))(x))
# { lambda ; a:f32[]. let
#     _:f32[] = sin a
#     b:f32[] = cos a
#     c:f32[] = mul 1.0 b
#   in (c,) }

{ lambda ; a:f32[]. let
    b:f32[] = mul a 1.5707963705062866
    c:f32[] = sin b
    d:f32[] = mul a 1.5707963705062866
    e:f32[] = sin d
    f:f32[] = mul c e
  in (f,) }
{ lambda ; a:f32[]. let
    b:f32[] = mul a 1.5707963705062866
    c:f32[] = sin b
    d:f32[] = cos b
    e:f32[] = mul a 1.5707963705062866
    f:f32[] = sin e
    g:f32[] = cos e
    _:f32[] = mul c f
    h:f32[] = mul c 1.0
    i:f32[] = mul 1.0 f
    j:f32[] = mul h g
    k:f32[] = mul j 1.5707963705062866
    l:f32[] = mul i d
    m:f32[] = mul l 1.5707963705062866
    n:f32[] = add_any k m
  in (n,) }


2023-05-04 04:52:44.284474: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:1182] failed to enqueue async memcpy from host to device: CUDA_ERROR_LAUNCH_FAILED: unspecified launch failure; GPU dst: 0x7f200e004b00; host src: 0x7f2433600200; size: 4=0x4
2023-05-04 04:52:44.284503: E external/xla/xla/stream_executor/stream.cc:336] Error recording event in stream: Error recording CUDA event: CUDA_ERROR_LAUNCH_FAILED: unspecified launch failure; not marking stream as bad, as the Event object may be at fault. Monitor for further errors.
2023-05-04 04:52:44.284514: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:617] unable to add host callback: CUDA_ERROR_LAUNCH_FAILED: unspecified launch failure
2023-05-04 04:52:44.284614: E external/xla/xla/stream_executor/cuda/cuda_event.cc:29] Error polling for event status: failed to query event: CUDA_ERROR_LAUNCH_FAILED: unspecified launch failure
2023-05-04 04:52:44.284631: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:698] could

In [ ]:

def diff_round(x):
    """
    differentiable version of round function
    """
    # return x - jnp.sin(2*jnp.pi*x)/(2*jnp.pi)
    # return jnp.sin(x*(jnp.pi/2))**2
    return jnp.sin(x*(jnp.pi/2))*jnp.sin(x*(jnp.pi/2))

def harder_diff_round(x):
    return diff_round(diff_round(x))

def differentiable_eq(a:float,b:float):
    """
    will give big value if a anb b are similar and small otherwise
    bot a and b are assumed to be between 0 and 1
    """
    a= harder_diff_round(a)
    b= harder_diff_round(b)
    res=a*b+(1-a)*(1-b)
    return harder_diff_round(res)

def differentiable_and(a:float,b:float):
    a= diff_round(a)
    b= diff_round(b)
    res=a*b
    return res

#versions with second entry keeping as int
v_differentiable_eq=jax.vmap(differentiable_eq,in_axes=(0,None))
v_v_differentiable_eq=jax.vmap(v_differentiable_eq,in_axes=(0,None))
v_v_v_differentiable_eq=jax.vmap(v_v_differentiable_eq,in_axes=(0,None))

def filter_mask_of_intrest(mask,initial_mask_id): #krowa test filter_mask_of_intrest
    """
    filters the mask to set to 1 only if it is this that we are currently intressted in 
    """
    coor_0_agree=v_v_differentiable_eq(mask[:,:,0],initial_mask_id[0])
    coor_1_agree=v_v_differentiable_eq(mask[:,:,1],initial_mask_id[1])
    coor_2_agree=v_v_differentiable_eq(mask[:,:,2],initial_mask_id[2])
    coor_3_agree=v_v_differentiable_eq(mask[:,:,3],initial_mask_id[3])
    a=differentiable_and(coor_0_agree,coor_1_agree)
    b=differentiable_and(coor_2_agree,coor_3_agree) 

    # coor_0_agree=v_v_differentiable_eq(mask[:,:,0],shift_x)
    # coor_1_agree=v_v_differentiable_eq(mask[:,:,1],shift_y)

    return differentiable_and(a,b)        

mask=jnp.array([[[0.,0.,0.,0.],[1.,0.,0.,0.],[0.,1.,0.,0.],[1.,1.,0.,0.]
                ,[0.,0.,1.,0.],[1.,0.,1.,0.],[0.,1.,1.,0.],[1.,1.,1.,0.]
                ,[0.,0.,0.,1.],[1.,0.,0.,1.],[0.,1.,0.,1.],[1.,1.,0.,1.]
                ,[0.,0.,1.,1.],[1.,0.,1.,1.],[0.,1.,1.,1.],[1.,1.,1.,1.]
                  ]])

shapp=mask.shape
print(shapp)
for i in range(shapp[1]):
    print(filter_mask_of_intrest(mask, mask[0,i,:]))

(1, 16, 4)
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [13]:
from super_voxels.SIN.SIN_jax_2D_simpler.model_sin_jax_2D import SpixelNet
from super_voxels.SIN.SIN_jax_2D_simpler.model_sin_jax_utils_2D import *
from super_voxels.SIN.SIN_jax_2D_simpler.shape_reshape_functions import *
from testUtils.spleenTest import get_spleen_data
import h5py
f = h5py.File('/workspaces/Jax_cuda_med/data/hdf5_loc/example_mask.hdf5', 'r+')
masks=f['masks'][:,:,:]
f.close()
cached_subj =get_spleen_data()[0]
masks= einops.rearrange(masks,'w h c->1 w h c')
masks.shape


data loaded from hdf5


(1, 256, 256, 4)

In [20]:
image=cached_subj[0][0,:,64:-64,64:-64,32:33]
image.shape

(1, 256, 256, 1)

In [21]:
cfg = config_dict.ConfigDict()
cfg.total_steps=300
# cfg.learning_rate=0.00002 #used for warmup with average coverage loss
cfg.learning_rate=0.0000001

cfg.num_dim=4
cfg.batch_size=160

cfg.batch_size_pmapped=np.max([cfg.batch_size//jax.local_device_count(),1])
cfg.img_size = (cfg.batch_size,1,256,256)
cfg.label_size = (cfg.batch_size,256,256)
cfg.r_x_total= 3
cfg.r_y_total= 3
cfg.orig_grid_shape= (cfg.img_size[2]//2**cfg.r_x_total,cfg.img_size[3]//2**cfg.r_y_total,cfg.num_dim)
cfg.masks_num= 4# number of mask (4 in 2D and 8 in 3D)

##getting the importance of the losses associated with deconvolutions
## generally last one is most similar to the actual image - hence should be most important
cfg.deconves_importances=(0.1,0.5,1.0)
#some constant multipliers related to the fact that those losses are disproportionally smaller than the other ones
cfg.edge_loss_multiplier=10.0
cfg.feature_loss_multiplier=10.0
#just for numerical stability
cfg.epsilon=0.0000000001 
cfg = ml_collections.FrozenConfigDict(cfg)
dynamic_cfg_a = config_dict.ConfigDict()
dynamic_cfg_a.is_beg=False
dynamic_cfg_a = ml_collections.config_dict.FrozenConfigDict(dynamic_cfg_a)


prng = jax.random.PRNGKey(42)
dim_stride=0
r_x=3
r_y=3
rearrange_to_intertwine_einops='f bb h w cc->bb (h f) w cc'
translation_val=4
features=32
initial_masks= jnp.stack([
            get_initial_supervoxel_masks(cfg.orig_grid_shape,0,0),
            get_initial_supervoxel_masks(cfg.orig_grid_shape,1,0),
            get_initial_supervoxel_masks(cfg.orig_grid_shape,0,1),
            get_initial_supervoxel_masks(cfg.orig_grid_shape,1,1)
                ],axis=0)

resized_image=image
curried=(resized_image,masks,masks,initial_masks)


model = De_conv_batched_for_scan(cfg,dynamic_cfg_a,dim_stride,r_x, r_y,rearrange_to_intertwine_einops,translation_val,features )
params = model.init({'params': prng}, curried,0)#['params']
new_curried,losses=model.apply(params, curried,0)


ValueError: Inconsistent batch axis sizes: {1, 4}